In [31]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [32]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import gradio as gr

In [33]:
description = """



Each column in the dataset is labeled and explained in more detail below. 

    manufacturer_name: the name of the car manufacturer 
    odometer_value: odometer state in kilometers 
    year_produced: the year the car was produced 
    engine_has_gas: whether or not the car has a propane tank with tubing 
    engine_type: the engine type of the car 
    engine_capacity: capacity of the engine in liters eg. 2.4
    has_warranty: whether the car has warranty 
    drivetrain: type of drivetrain (front, rear, all) 
    duration_listed: the number of days the car is listed in the catalog 


"""

In [34]:
# Ignore Warnings 

import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [35]:
# Importing the training data

df_train = pd.read_csv("public_cars.csv")

In [36]:
#df_train.columns
#df_train.info()

In [37]:
def process_df(data_f):
    
    one_hot_variables = ['manufacturer_name', 'drivetrain']
    data_f.dropna(inplace = True)
    data_f = data_f[['manufacturer_name', 'year_produced', 'odometer_value', 'engine_has_gas', 'engine_capacity', 'has_warranty', 'drivetrain', 'duration_listed', 'price_usd']]
    data_f['engine_has_gas'] = data_f['engine_has_gas'].map({False: 0, True: 1})
    data_f['has_warranty'] = data_f['has_warranty'].map({False: 0, True: 1})
    data_f = pd.get_dummies(data_f, columns = one_hot_variables)
    return data_f

In [38]:
df_train = process_df(df_train)

/Users/rohan10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/rohan10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [39]:
X = df_train.loc[:, df_train.columns != 'price_usd']
y = df_train.loc[:, df_train.columns == 'price_usd']
X = X.values
y = y.values
# Min Max Scaling
#scaler = MinMaxScaler()
#X_train = scaler.fit_transform(X)

In [40]:
reg_rand = RandomForestRegressor(n_estimators = 1000)
reg_rand.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [41]:
def price_car(manufacturer_name, year_produced, odometer_value, engine_has_gas, engine_capacity, has_warranty, drivetrain, duration_listed):
    
    df = pd.DataFrame(list(zip([manufacturer_name], [year_produced], 
            [odometer_value],
            [engine_has_gas],
            [engine_capacity],
            [has_warranty],
            [drivetrain],
            [duration_listed])), columns= ["manufacturer_name", "year_produced", "odometer_value","engine_has_gas", "engine_capacity", "has_warranty","drivetrain","duration_listed"])
    df['engine_has_gas'] = df['engine_has_gas'].map({False: 0, True: 1})
    df['has_warranty'] = df['has_warranty'].map({False: 0, True: 1})
    one_hot_variables = ['manufacturer_name', 'drivetrain']
    df_testing = pd.get_dummies(df, columns = one_hot_variables)
    global df_train
    df_train, df_testing = df_train.align(df_testing, join = 'left', axis = 1)
    df_testing = df_testing.loc[:, df_testing.columns != 'price_usd']
    
    for i in df_testing.columns:
        
        df_testing[i] = df_testing[i].replace(np.nan, 0)
    
    prediction = reg_rand.predict(df_testing)[0].round()
    return prediction



manufacturer_name = "Audi"
year_produced = 2018
odometer_value = 10
engine_has_gas = True
engine_capacity = 3.0
has_warranty = True
drivetrain = "rear"
duration_listed = 1




test = price_car(manufacturer_name, year_produced,
          odometer_value, 
          engine_has_gas, 
          engine_capacity, 
          has_warranty, 
          drivetrain, 
          duration_listed)
test

In [42]:
df_cols = pd.read_csv("public_cars.csv")

In [43]:
manufacture_choices = df_cols["manufacturer_name"].unique().tolist()
yr_produced = df_cols["year_produced"].unique().tolist()
yr_produced = sorted(yr_produced)
drive_train = df_cols["drivetrain"].unique().tolist()

In [44]:
# Building input interface


manufacturer_name_input = gr.inputs.Dropdown(choices = manufacture_choices, 
                                             type = "value", 
                                             label = "Manufacture Name")

year_produced_input = gr.inputs.Dropdown(choices = yr_produced, 
                                             type = "value", 
                                             label = "Year Produced")

odometer_value_input = gr.inputs.Number(label = "Odometer Value in Kms")

engine_has_gas_input = gr.inputs.Checkbox(label = "Engine has gas?")

engine_capacity_input = gr.inputs.Number(label = "Engine Capacity?")

has_warranty_input = gr.inputs.Checkbox(label = "Has Warranty?")

drivetrain_input = gr.inputs.Dropdown(choices = drive_train, 
                                             type = "value", 
                                             label = "Drivetrain")

duration_listed_input = gr.inputs.Number(label = "Number of Days Listed?")

In [45]:
# Compiling entire interface

iface = gr.Interface(price_car, [manufacturer_name_input, year_produced_input, odometer_value_input, 
                                engine_has_gas_input, engine_capacity_input, has_warranty_input,
                                drivetrain_input, duration_listed_input], 
                    "number",
                    title = "Car Price Prediction",
                    article = description)

In [46]:
iface.launch(share = True)

Running locally at: http://127.0.0.1:7865/
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://28787.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7865/',
 'https://28787.gradio.app')

Exception in thread Thread-1592:
Traceback (most recent call last):
  File "/Users/rohan10/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/Users/rohan10/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/gradio/tunneling.py", line 31, in handler
    r, w, x = select.select([sock, chan], [], [])
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/paramiko/channel.py", line 938, in fileno
    self._pipe = pipe.make_pipe()
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/paramiko/pipe.py", line 35, in make_pipe
    p = PosixPipe()
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/paramiko/pipe.py", line 43, in __init__
    self._rfd, self._wfd = os.pipe()
OSError: [Errno 24] Too many open files
Unhandled exception in thread started by <bound method Thread._bootstrap of <Thread(Thread-1592, started daemo

Error on request:
Traceback (most recent call last):
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packa

Error on request:
Traceback (most recent call last):
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packa

...

Error on request:
Traceback (most recent call last):
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packa

Error on request:
Traceback (most recent call last):
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packa

Error on request:
Traceback (most recent call last):
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packa

Error on request:
Traceback (most recent call last):
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packa

Error on request:
Traceback (most recent call last):
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/rohan10/anaconda3/lib/python3.7/site-packa